In [1]:
import sys
import nltk
from nltk import word_tokenize, sent_tokenize
from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
# nltk.download('stopwords')
# from nltk.tokenize.punkt import PunktWordTokenizer
import gensim
from gensim.models.word2vec import Word2Vec
from gensim.models import FastText
import re
from sklearn.manifold import TSNE
from sklearn import svm
from sklearn.model_selection import cross_val_score
import pandas as pd
import numpy as np
from bokeh.io import output_notebook
from bokeh.plotting import show, figure
from string import digits
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.manifold        
import enchant
from enchant.checker import SpellChecker
import xgboost as xgb
from sklearn import metrics

In [2]:
# read data
path ='./'
dataset = pd.read_csv(path + 'train.csv'.format(1), engine='python')

In [3]:
word = 'toxic'
print(dataset[word][dataset[word] == 0].value_counts())
print(dataset[word][dataset[word] == 1].value_counts())

0    144277
Name: toxic, dtype: int64
1    15294
Name: toxic, dtype: int64


In [25]:
15294*9

137646

In [ ]:
from sklearn.utils import resample

# Separate majority and minority classes
dataset_majority = dataset[dataset.toxic == 0]
dataset_minority = dataset[dataset.toxic == 1]
 
# # Upsample minority class
# dataset_minority_upsampled = resample(dataset_minority, 
#                                  replace = True,  # sample with replacement
#                                  n_samples = 144277,   # to match majority class
#                                  random_state = 123) # reproducible results
 
# # Combine majority class with upsampled minority class
# dataset_upsampled = pd.concat([dataset_majority, dataset_minority_upsampled])
 
# # Display new class counts
# dataset_upsampled.toxic.value_counts()

# Downsample majority class
dataset_majority_downsampled = resample(dataset_majority, replace = True, n_samples = len(dataset_minority),
                                        random_state = 123)

# Combine two classes
dataset_ds = pd.concat([dataset_minority, dataset_majority_downsampled])

# Shuffle dataset
dataset_ds = dataset_ds.sample(frac=1).reset_index(drop=True)

In [11]:
ds=dataset[dataset['toxic']==1]

In [ ]:
!pip install joblib textblob -U

In [4]:
from joblib import Parallel, delayed
from textblob import TextBlob
from textblob.translate import NotTranslated

import argparse
import os
import pandas as pd

In [5]:
NAN_WORD = "_NAN_"


def translate(comment, language):
    if hasattr(comment, "decode"):
        comment = comment.decode("utf-8")

    text = TextBlob(comment)
    try:
        text = text.translate(to='de')
        text = text.translate(to=language)
        text = text.translate(to="en")
    except NotTranslated:
        pass

    return str(text)



In [23]:
dp=ds[["comment_text", "toxic"]][0:100]
df = ds[["comment_text", "toxic"]][0:100].copy()

def upsample(comment, label):
    global dp
    l = ['es', 'fr', 'it']
    for language in l:
        new_comment = translate(comment, language)
        dp=dp.append(pd.Series({"comment_text":new_comment,"toxic":label}),ignore_index=True)
        

df.apply(lambda x: upsample(x["comment_text"], x["toxic"]), axis=1)

6      None
12     None
16     None
42     None
43     None
44     None
51     None
55     None
56     None
58     None
59     None
65     None
79     None
86     None
105    None
151    None
159    None
168    None
176    None
181    None
201    None
206    None
211    None
218    None
231    None
238    None
268    None
278    None
286    None
295    None
       ... 
687    None
698    None
700    None
718    None
722    None
726    None
730    None
732    None
737    None
746    None
756    None
761    None
802    None
806    None
807    None
815    None
817    None
827    None
830    None
852    None
853    None
877    None
884    None
887    None
888    None
925    None
931    None
934    None
952    None
957    None
Length: 100, dtype: object

In [24]:
dp

,comment_text,toxic
0,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
1,Hey... what is it..\n@ | talk .\nWhat is it......,1
2,"Bye! \n\nDon't look, come or think of comming ...",1
3,You are gay or antisemmitian? \n\nArchangel WH...,1
4,"FUCK YOUR FILTHY MOTHER IN THE ASS, DRY!",1
5,I'm Sorry \n\nI'm sorry I screwed around with ...,1
6,GET FUCKED UP. GET FUCKEEED UP. GOT A DRINK T...,1
7,Stupid peace of shit stop deleting my stuff as...,1
8,=Tony Sidaway is obviously a fistfuckee. He lo...,1
9,My Band Page's deletion. You thought I was gon...,1


In [25]:
tokenizer = TreebankWordTokenizer()
# print(tokenizer.tokenize(dataset_ds['comment_text'].iloc[0]))
chkr = SpellChecker("en_UK", "en_US")

def low(x):
#     x = x.decode('utf-8')
#     if type(x) != unicode:
#         x = unicode(x)
    # remove numbers
#     filter(lambda x: x.isalpha(), x)
#     x = re.sub(r'[^\w]', ' ', x)

    # spelling checker
    x = re.sub(r'[\W_]+', ' ', x)
    chkr.set_text(x)
    for err in chkr:
#         print(err.suggest())
        if len(err.suggest()) > 0:
            sug = err.suggest()[0]
            err.replace(sug)
            x = chkr.get_text()
    
    return x.lower()

def tok_and_rem(x):
#     if type(x) != unicode:
#         x = unicode(x)
#     s = word_tokenize(x)
    s = x.split()
    l = list(s)
    for word in s:
        if (len(word) <= 2):
            l.remove(word)
        elif word in stopwords.words('english'):
            l.remove(word)
    return l

# to lowercase and remove special characters
# dataset_ds['comment_text'] = dataset_ds['comment_text'].apply(lambda x: low(x))
dp['comment_text'] = dp['comment_text'].apply(lambda x: low(x))

# tokenize and remove stopwords
dp['comment_text'] = dp['comment_text'].apply(lambda x: tok_and_rem(x))


In [33]:
print(dp["comment_text"].iloc[1])
print(dp["comment_text"].iloc[102])
print(dp["comment_text"].iloc[103])


['hey', 'talk', 'exclusive', 'group', 'tali', 'bans', 'good', 'destroying', 'self', 'appointed', 'purist', 'gang', 'one', 'asks', 'questions', 'abet', 'anti', 'social', 'destructive', 'non', 'contribution', 'ask', 'sottish', 'clean', 'behavior', 'issue', 'nonsensical', 'warnings']
['hey', 'talk', 'exclusive', 'group', 'taliban', 'good', 'destroying', 'destructive', 'self', 'proclaimed', 'purists', 'anyone', 'provides', 'questions', 'antisocial', 'destroy', 'understand', 'contribution', 'ask', 'sottish', 'clean', 'behavior', 'instead', 'warning', 'pointless']
['hey', 'speak', 'exclusive', 'group', 'taliban', 'good', 'destroying', 'destructive', 'self', 'proclaimed', 'purists', 'anybody', 'provides', 'questions', 'antisocial', 'destruction', 'contribution', 'understand', 'ask', 'sottish', 'purify', 'behavior', 'instead', 'giving', 'unnecessary', 'warnings']


In [ ]:
print(dataset[(dataset["severe_toxic"] == 1) & (dataset["toxic"] == 1)].count())
print(dataset[dataset["severe_toxic"] == 1].count())

In [ ]:



parser = argparse.ArgumentParser("Script for extending train dataset")
parser.add_argument("train_file_path")
parser.add_argument("--languages", nargs="+", default=["es", "de", "fr"])
parser.add_argument("--thread-count", type=int, default=300)
parser.add_argument("--result-path", default="extended_data")

args = parser.parse_args()

train_data = pd.read_csv(args.train_file_path)
comments_list = train_data["comment_text"].fillna(NAN_WORD).values

if not os.path.exists(args.result_path):
    os.mkdir(args.result_path)

parallel = Parallel(args.thread_count, backend="threading", verbose=5)
for language in args.languages:
#     print('Translate comments using "{0}" language'.format(language))
    translated_data = parallel(delayed(translate)(comment, language) for comment in comments_list)
    train_data["comment_text"] = translated_data
    print(train_data.head())

#     result_path = os.path.join(args.result_path, "train_" + language + ".csv")
# train_data.to_csv(result_path, index=False)